# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to have you here. How can I assist you today?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://saisamarth.me")
print(ed.title)
print(ed.text)

SaiSamarth – Welcome to World of SaiSamarth
SaiSamarth
Menu
SaiSamarth
Menu
Home
About
Resume
Portfolio
Contact
Shortcodes
Social
SCROLL
INTRODUCTION
I'm a Student interested in
Artificial Intelligence
ABOUT
I'm a passionate student with a keen interest in the fascinating world of Artificial Intelligence.
Currently pursuing my studies in Computer Science, I have dedicated my time to exploring the realms of AI and Data Science. From developing machine learning models to creating predictive analytics solutions, I thrive on leveraging the power of data to solve real-world problems. As an enthusiastic learner, I continuously seek to expand my knowledge and stay updated with the latest advancements in AI.
PORTFOLIO
Selected Projects
I invite you to explore my portfolio, where you can find a collection of my projects showcasing my skills and passion for this dynamic field.
DragGAN – Using Real Images
Surface Experience
Surface Experience
Lorem Ipsum is simply dummy text of the printing and t

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [12]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [13]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [14]:
print(user_prompt_for(ed))

You are looking at a website titled SaiSamarth – Welcome to World of SaiSamarth
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

SaiSamarth
Menu
SaiSamarth
Menu
Home
About
Resume
Portfolio
Contact
Shortcodes
Social
SCROLL
INTRODUCTION
I'm a Student interested in
Artificial Intelligence
ABOUT
I'm a passionate student with a keen interest in the fascinating world of Artificial Intelligence.
Currently pursuing my studies in Computer Science, I have dedicated my time to exploring the realms of AI and Data Science. From developing machine learning models to creating predictive analytics solutions, I thrive on leveraging the power of data to solve real-world problems. As an enthusiastic learner, I continuously seek to expand my knowledge and stay updated with the latest advancements in AI.
PORTFOLIO
Selected Projects
I invite you to explore my portfolio, where you can find 

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [17]:
messages = [
    {"role": "system", "content": "You are a 5 year old kid."},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [18]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

2 + 2 is 4! Yay!


## And now let's build useful messages for GPT-4o-mini, using a function

In [19]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [20]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled SaiSamarth – Welcome to World of SaiSamarth\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSaiSamarth\nMenu\nSaiSamarth\nMenu\nHome\nAbout\nResume\nPortfolio\nContact\nShortcodes\nSocial\nSCROLL\nINTRODUCTION\nI\'m a Student interested in\nArtificial Intelligence\nABOUT\nI\'m a passionate student with a keen interest in the fascinating world of Artificial Intelligence.\nCurrently pursuing my studies in Computer Science, I have dedicated my time to exploring the realms of AI and Data Science. From developing machine learning models to creating predictive analytics solutions, I thrive on leveraging the po

## Time to bring it together - the API for OpenAI is very simple!

In [21]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [22]:
summarize("https://saisamarth.me")

'# Summary of SaiSamarth Website\n\n**Introduction**\nSaiSamarth is a personal website presenting the work and interests of a Computer Science student with a focus on Artificial Intelligence. The site showcases various projects and skills related to AI and Data Science.\n\n**About**\nThe individual describes their passion for AI, highlighting their experience in developing machine learning models and predictive analytics solutions.\n\n**Portfolio**\nThe portfolio features selected projects, including:\n- **DragGAN** – Using Real Images\n- **Surface Experience**\n- **Adaptive Generation**\n- **Frame Manufacturer**\n\nEach project aims to demonstrate their skills in the AI domain.\n\n**Services**\nThe website outlines services offered, including:\n- **Design:** Brand Systems, Interaction Design, Motion Design\n- **Marketing:** Strategy and Analytics\n- **Development:** UI Development, App Development, Cloud Infrastructure\n\n**Resume**\nThe resume section includes prior experiences worki

In [23]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [24]:
display_summary("https://saisamarth.me")

# SaiSamarth Website Summary

**Overview:**  
SaiSamarth is a personal portfolio website belonging to a student passionate about Artificial Intelligence and Data Science. The website showcases the individual's skills, projects, and services while providing insight into their achievements and experience in the tech field.

## About
The creator is a dedicated student pursuing Computer Science, with a focus on AI and Data Science. They engage in various projects involving machine learning and predictive analytics, showcasing a commitment to leveraging data for real-world problem-solving.

## Portfolio
The portfolio features a selection of projects, although detailed descriptions are limited. Notable projects include:
- **DragGAN:** Utilizing real images.
- **Surface Experience:** (Details not provided)
- **Adaptive Generation:** (Details not provided)
- **Frame Manufacturer:** (Details not provided)

## Services
The site outlines a range of services offered:
- **Design:** Visual identities, interaction design, and motion design.
- **Marketing:** Strategies, storytelling, and analytics.
- **Development:** UI/Back-End development and consultancy.

## Resume
The creator's work experience includes:
- **Google** (2017-2019): Full-Stack Developer
- **Tesla** (2015-2017): Front-End Developer
- **Adobe** (2014-2015): Javascript Developer

## Skills and Achievements
Skills highlighted include HTML/CSS, Javascript, and WordPress with various proficiency percentages. Achievements boast:
- 40 Awards
- 30+ Apps Developed
- 90+ Projects Completed
- 130% Client ROI

## Testimonials
Customer feedback emphasizes the quality and efficiency of services provided, with noted clients including Tesla and Ubisoft Montreal.

## Pricing
The website contains a pricing section offering:
- **Full-Time Rate:** $59 per hour
- **Contract Rate:** $79 per hour

## Contact
Visitors can reach out through a contact form provided for inquiries and further communication. 

Overall, the website encapsulates the creator's journey in AI, their creative endeavors, and their professional services in technology and design.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [25]:
display_summary("https://cnn.com")

# CNN Summary

CNN is a prominent news organization that provides up-to-date news coverage on a variety of topics, including US and world news, politics, business, health, entertainment, science, and sports. The site also features live videos, special reports, and various sections dedicated to specific themes like climate, politics, and travel.

## Key News Announcements:
- **Israel-Hamas Conflict**: Israel has approved a ceasefire and hostage deal, with a ceasefire expected to begin after intense cabinet discussions.
- **TikTok**: The platform announced it would be unavailable in the US unless immediate government intervention occurs.
- **Russia in Africa**: An analysis highlights Russia's expanding influence in Africa amid geopolitical tensions.

The site also includes features on current events, such as significant political moves, environmental issues, and sports news, maintaining a focus on in-depth analysis and ongoing reporting from various correspondents.

In [26]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic is an AI safety and research company based in San Francisco, focused on creating reliable and beneficial AI systems. The website features information about their main product, Claude, including the latest model updates.

## Key Offerings:
- **Claude 3.5 Sonnet**: The newest and most intelligent AI model available for user interactions.
- **Claude for Enterprise**: Tailored AI solutions for businesses.
- **API Access**: Tools to build AI-powered applications and custom experiences.

## Recent Announcements:
- **October 22, 2024**: Introduced new features for Claude 3.5 Sonnet and a new model, Claude 3.5 Haiku.
- **September 4, 2024**: Further developments related to Claude for Enterprise.
- **Previous Updates**: Research on AI safety and alignment, including insights on "Constitutional AI" from December 2022, and discussions on AI safety principles from March 2023.

Anthropic emphasizes a multidisciplinary approach, pooling expertise from various fields to enhance AI safety and functionality.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# !pip install PyPDF2

In [29]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    try:
        # Open the PDF file in read-binary mode
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            # Iterate through all the pages
            for page in reader.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        return f"An error occurred: {e}"

# Specify the path to your PDF file
pdf_file_path = "../CV.pdf"

# Extract and print the text
extracted_text = extract_text_from_pdf(pdf_file_path)
print(extracted_text)


SAISAMARTH  TALURI  
07552884811  ⋄ saisamarth@gmail.com  ⋄ linkedin.com  ⋄ github.com  
 
 
PERSONAL  STATEMENT   
Computer science graduate advancing my expertise through a master’s in Speech and Language Processing at Edinburgh. Extensive  AI 
experience across internships, research, and projects, with proficiency in machine learning, deep learning, and LLMs.   
EDUCATION   
MSc  Speech  and Language  Processing , University  of Edinburgh  Sept  2024  — Present  
• Awarded  India  Merit  Scholarship  of 5000  pounds  
• Core  modules  include:  Speech  Processing,  Natural  Language  Processing,  Applied  Machine  Learning  
BTech  Computer  Science  and Engineering , PES University  Aug 2020  — May  2024  
CGPA:  9.12  
• Graduated  with  a specialization  in Machine  Intelligence  and Data  science  
• Awarded  scholarships  for academic excellence  in 1nd and 2rd year  
• Chosen  specialism  modules  include:  Machine  Learning,  Deep  Learning,  Computer  Vision,  Data  Analytic

In [31]:
# Step 1: Create your prompts

system_prompt = "You are a HR manager who analyses CVs and finds the best type of role for that particular CV."
user_prompt = """
    Given below is the content of a CV. Analyse the CV and give a short paragraph on which roles the CV profile is best suited for. 
    CV Content: \n\n
"""
user_prompt +=  extract_text_from_pdf(pdf_file_path)
# Step 2: Make the messages list

messages = [{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )

# Step 4: print the result

print(response.choices[0].message.content)

Saisamarth Taluri's CV indicates a strong background in computer science, specifically in artificial intelligence, machine learning, and speech and language processing. Given his ongoing master's studies at the University of Edinburgh and extensive hands-on experience in developing AI-driven solutions, he is well-suited for roles such as AI/Machine Learning Engineer, Data Scientist, or NLP Engineer. His experience with model fine-tuning, data pipeline engineering, and projects involving large language models makes him particularly qualified for positions that require expertise in natural language processing and machine learning applications in various domains, including healthcare and finance. Additionally, his research contributions and publications enhance his profile for roles that require a blend of technical and theoretical understanding in AI technologies.


## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293